In [1]:
import underworld3 as uw
import petsc4py
from petsc4py import PETSc

In [23]:
import sympy
from IPython.display import Markdown, Latex, display

mesh2 = uw.meshing.UnstructuredSimplexBox(minCoords=(0.0,0.0), 
                                              maxCoords=(1.0,1.0), 
                                              cellSize=1.0/12.0, 
                                              regular=True)

u2 = uw.discretisation.MeshVariable(r"\mathbf{u2}", mesh2, mesh2.dim, vtype=uw.VarType.VECTOR, degree=2)
p2 = uw.discretisation.MeshVariable(r"p^{(2)}", mesh2, mesh2.dim, vtype=uw.VarType.SCALAR, degree=1)
phi2 = uw.discretisation.MeshVariable(r"\phi^{(2)}", mesh2, 1, vtype=uw.VarType.SCALAR, degree=2)
dis2 = uw.discretisation.MeshVariable(r"\lambda^{(2)}", mesh2, 1, vtype=uw.VarType.SCALAR, degree=0, continuous=True )

Cmodv = uw.systems.constitutive_models.ViscousFlowModel(2)

In [24]:
coord_dm = mesh2.dm.getCoordinateDM()


In [25]:
clone_dm = mesh2.dm.clone()

degree=1

options = PETSc.Options()
options.setValue("fe_petscspace_degree", degree)
options.setValue("fe_petscdualspace_lagrange_continuity", True)

petsc_fe = PETSc.FE().createDefault(mesh2.dim, mesh2.dim, mesh2.isSimplex, degree, "fe_", PETSc.COMM_WORLD)

In [26]:
clone_dm.setField(0, petsc_fe)
clone_dm.createDS()

In [27]:
matInterp, vecScale = coord_dm.createInterpolation(clone_dm)
coordsOld  = mesh2.dm.getCoordinates()
coordsNewL = clone_dm.getLocalVec()
coordsNewG = matInterp * coordsOld
clone_dm.globalToLocal(coordsNewG, coordsNewL)


In [28]:
coordsNewL.array.reshape(-1,2).shape

(169, 2)

In [29]:
dis2.petsc_fe.view()

PetscFE Object: P0 (\lambda^{(2)}_) 1 MPI process
  type: basic
  Basic Finite Element in 2 dimensions with 1 components
  PetscSpace Object: P0 (\lambda^{(2)}_) 1 MPI process
    type: poly
    Space in 2 variables with 1 components, size 1
    Polynomial space of degree 0
  PetscDualSpace Object: P0 (\lambda^{(2)}_) 1 MPI process
    type: lagrange
    Dual space with 1 components, size 1
    Discontinuous Lagrange dual space
    Quadrature of order 1 on 1 points (dim 2)


In [32]:
dis2.coords[0:10,:]

array([[0.02777778, 0.02777778],
       [0.05555556, 0.05555556],
       [0.05555556, 0.11111111],
       [0.02777778, 0.13888889],
       [0.02777778, 0.19444444],
       [0.05555556, 0.22222222],
       [0.05555556, 0.27777778],
       [0.02777778, 0.30555556],
       [0.02777778, 0.36111111],
       [0.05555556, 0.38888889]])

In [31]:
# Note, the discontinuous nodes overlie the continuous ones but there are more of them !

if uw.mpi.size==1:

    import numpy as np
    import pyvista as pv
    import vtk

    pv.global_theme.background = 'white'
    pv.global_theme.window_size = [750, 250]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = 'panel'
    pv.global_theme.smooth_shading = True
        
    mesh2.vtk("tmp_mesh.vtk")
    pvmesh = pv.read("tmp_mesh.vtk")
    
    # coords2D = coordsNewL.array.reshape(-1,2)
    coords2D = dis2.coords
    coords = np.zeros((coords2D.shape[0],3))
    coords[:,0:2] = coords2D[:,:]
    points = pv.PointSet(coords)

    pl = pv.Plotter()
    pl.add_mesh(pvmesh,'Black', 'wireframe')
    pl.add_points(points, color="Red", render_points_as_spheres=False, point_size=3.0)

    pl.show(cpos="xy")